In [ ]:
# default_exp core
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from IPython.display import display, SVG

# Notes

> Extra documentation for `jacobs-vault`.

Mostly Markdown.


In [ ]:
#hide
#from nbdev.showdoc import *

## Satellite Coverage Libraries

Considered:
* pyEphem, but the inverse calcs seem to be broken.
* pysat may be of interest, esp. pysat.Orbits
* pyOrbital, though for coordinate transform it is included in Skyfield.

Selected: **Skyfield**, "Elegant astronomy for Python" that uses py-sgp4 for speed. Includes example for [Loading a TLE file](https://rhodesmill.org/skyfield/earth-satellites.html#loading-a-tle-file):
  <blockquote>
  But the most popular approach is to ask the topocentric position for its 
  altitude and azimuth coordinates, which tell you whether the satellite is above or 
  below the horizon:

  ```python
  alt, az, distance = topocentric.altaz()

  if alt.degrees > 0:
    print('The ISS is above the horizon')

  print(alt)
  print(az)
  print(int(distance.km), 'km')
  The ISS is above the horizon
  16deg 16' 32.6"
  350deg 15' 20.4"
  1168 km
  ```
  </blockquote>
  
For example, the following loads (old) ISS elements, and asks about rise/set times, and if it is now above the horizon.

In [ ]:
from skyfield.api import EarthSatellite, Topos, load

ts = load.timescale()

# Old ISS data
line1 = '1 25544U 98067A   14020.93268519  .00009878  00000-0  18200-3 0  5082'
line2 = '2 25544  51.6498 109.4756 0003572  55.9686 274.8005 15.49815350868473'
satellite = EarthSatellite(line1, line2, 'ISS (ZARYA)', ts)

# Sample location
bluffton = Topos('40.8939 N', '83.8917 W')

# Relative vector
difference = satellite - bluffton
t0 = ts.utc(2017, 1, 23)
t1 = ts.utc(2017, 1, 24)
topocentric = difference.at(t0)
alt, az, distance = topocentric.altaz()

# Quick report for t0
if alt.degrees > 0:
    print('The ISS is above the horizon')
print(alt, az, int(distance.km), 'km')

# When does it rise and set, between t0 and t1, for this location?
times, events = satellite.find_events(bluffton, t0, t1 , altitude_degrees=30.0)
for ti, event in zip(times, events):
    name = ('rise above 30°', 'culminate', 'set below 30°')[event]
    print(ti.utc_strftime('%Y %b %d %H:%M:%S'), name)


-42deg 16' 16.3" 248deg 33' 15.6" 9008 km
2017 Jan 23 05:06:28 rise above 30°
2017 Jan 23 05:07:31 culminate
2017 Jan 23 05:08:35 set below 30°
2017 Jan 23 22:56:38 rise above 30°
2017 Jan 23 22:57:13 culminate
2017 Jan 23 22:57:48 set below 30°


### Other Resources
* Interactive web page, [Satellite beam design](https://www.satsig.net/satellite/satellite-beam-design.htm)
Plots rough ellipses, but seems to be embedded js, not RESTful. Uses Google for the map.

* Alan Washburn (NPS), [Earth Coverage by Satellites in Circular Orbit](https://faculty.nps.edu/awashburn/Files/Notes/EARTHCOV.pdf)


* Bieliech et al 2019: [A study into satellite coverage...](https://clas.ucdenver.edu/math-clinic/sites/default/files/attached-files/ball_final_report.pdf)
"Math Clinic" capstone project, sponsored by Ball Aerospace. We have permission to use their jupyter notebook, which does the Kepler orbit calculations from scratch. However, while we could use it to check other solutions, we don't have redistribution rights, so it shouldn't go in the repo or solution.  See also [Commentary on Bielich et al. by Prof. Fournier](https://clas.ucdenver.edu/math-clinic/sites/default/files/attached-files/ball_final_report.pdf) for limitations.

* Cakaj et al [The coverage analysis for LEO satellites at low elevation](https://thesai.org/Downloads/Volume5No6/Paper_2-The_Coverage_Analysis_for_Low_Earth_Orbiting_Satellites_at_Low_Elevation.pdf)
IJACSA 5:6, 2014.

* Bettner, [Tools for Satellite Ground Track and Coverage Analysis](https://apps.dtic.mil/dtic/tr/fulltext/u2/a286848.pdf) via DTIC. 1995 report for USAF, software in Microsoft Excel(!!). Gives Visual Basic code in the appendix.


### Alan Goldberg of NOVAC suggests: 

>If your application is to determine satellite FOR, I suggest 
https://www.taitussoftware.com/products/applications/savoir/.  ...
> It was easier to use than either STK or government proprietary software.

However, I think it's commercial proprietary, so no go. He continues:

>Other options, which I have not used myself:
> 
> I suspect you will find what you need in this 
> http://celestrak.com/publications/AIAA/2006-6753/
> 
> Here is a matlab routine 
> https://in.mathworks.com/matlabcentral/fileexchange/55698-satellite-ephemeris-to-lla-eph2lla-m
> 
> JPL’s HORIZON can do everything, but may not be convenient 
> https://ssd.jpl.nasa.gov/?horizons_doc
> 
> If you roll your own, I’d suggest first changing the satellite from TLE (which is 
in ECI frame) to cartesian in the ECEF(WGS-84) frame, then converting to geodetic 
or whatever other system you wish.